<a href="https://colab.research.google.com/github/SamAlrahmani/DS-Unit-1-Sprint-2-Statistics/blob/master/Sam_Alrahmani_LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

In [96]:
### MY CODE STARTS HERE
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import ttest_1samp, ttest_ind, ttest_ind_from_stats, ttest_rel

v= pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data',
                    header=None,names=['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']) 
v


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,republican,n,n,y,y,y,y,n,n,y,y,n,y,y,y,n,y
431,democrat,n,n,y,n,n,n,y,y,y,y,n,n,n,n,n,y
432,republican,n,?,n,y,y,y,n,n,n,n,y,y,y,y,n,y
433,republican,n,n,n,y,y,y,?,?,?,?,n,y,y,y,n,y


In [97]:
# replacing the unknown values with NaN, and replacing yes and no with 0 & 1
# because we can't graph strings. we can only graph numbers. 
v = v.replace({'?':np.NaN, 'n':0, 'y':1})
v.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [98]:
# checking the empty data sets. 
v.isnull().sum()


party                     0
handicapped-infants      12
water-project            48
budget                   11
physician-fee-freeze     11
el-salvador-aid          15
religious-groups         11
anti-satellite-ban       14
aid-to-contras           15
mx-missile               22
immigration               7
synfuels                 21
education                31
right-to-sue             25
crime                    17
duty-free                28
south-africa            104
dtype: int64

In [99]:
# splitting the data to democrats and republican data sets. 
rep = v[v['party']=='republican']
dem = v[v['party']=='democrat']
print('rep:', rep.shape)
print('dem:', dem.shape)

rep: (168, 17)
dem: (267, 17)


In [100]:
#getting the mean of each data
print('rep mean: ', rep['crime'].mean())
print('dem mean: ', dem['crime'].mean())

rep mean:  0.9813664596273292
dem mean:  0.35019455252918286


In [101]:
#create artificial data !

a=0 # mean 
b=1 #std
sample = np.random.normal(a, b, 1000)
sample[:10]

array([-1.4901223 ,  0.27519357, -0.96724462,  1.1725405 , -0.25743812,
        0.73132197, -1.68007346, -1.23362729,  0.04002806,  1.92703564])

In [102]:
#I'm try to create the following function as a replicate of the lecture codes. 
# a = sample one, b = null hypothesis, omit nan values
def one_sample_ttest(a,b,nan_policy='omit'):
  return ttest_1samp(a,b,nan_policy='omit')
  # a = sample one, b = sample 2, omit nan values
def two_sample_ttest(a,b,nan_policy='omit'):
  return ttest_ind(a,b,nan_policy='omit')

  one_sample_ttest(rep['water-project'], 1)
two_sample_ttest(rep['education'], dem['education'])


Ttest_indResult(statistic=20.500685724563073, pvalue=1.8834203990450192e-64)

In [103]:
v

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,republican,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
431,democrat,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
432,republican,0.0,NaN,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
433,republican,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,0.0,1.0,1.0,1.0,0.0,1.0


In [106]:
#combine all the features that the rep and dem have it to get back through it and find the answer we looking for 
rep_data = [rep['handicapped-infants'], rep['physician-fee-freeze'], 
           rep['water-project'], rep['budget'],
           rep['el-salvador-aid'], rep['religious-groups'],
           rep['anti-satellite-ban'], rep['aid-to-contras'],
           rep['mx-missile'], rep['immigration'],
           rep['synfuels'], rep['education'],
           rep['right-to-sue'], rep['crime'],
           rep['duty-free'], rep['south-africa']]

dem_data = [dem['handicapped-infants'], dem['physician-fee-freeze'], 
           dem['water-project'], dem['budget'],
           dem['el-salvador-aid'], dem['religious-groups'],
           dem['anti-satellite-ban'], dem['aid-to-contras'],
           dem['mx-missile'], dem['immigration'],
           dem['synfuels'], dem['education'],
           dem['right-to-sue'], dem['crime'],
           dem['duty-free'], dem['south-africa']]
# I test the lists and it could work togather combine with no need to 
#individually write them

#Null hypothesis of 0.5 for the republican's data
[one_sample_ttest(a, 0.5) for a in rep_data]
#AND THE LIST LOOKS WORKING GOOD !

[Ttest_1sampResult(statistic=-10.232833482397659, pvalue=2.572179359890009e-19),
 Ttest_1sampResult(statistic=57.09643655679979, pvalue=3.8884659739684535e-110),
 Ttest_1sampResult(statistic=0.16385760607458383, pvalue=0.8700683158522193),
 Ttest_1sampResult(statistic=-13.705331355148527, pvalue=6.249973784238298e-29),
 Ttest_1sampResult(statistic=26.920515819250607, pvalue=4.531756691646779e-62),
 Ttest_1sampResult(statistic=16.844895175868118, pvalue=1.103043623086e-37),
 Ttest_1sampResult(statistic=-7.694446231812848, pvalue=1.3430544790393879e-12),
 Ttest_1sampResult(statistic=-12.048344034968558, pvalue=4.758689364006887e-24),
 Ttest_1sampResult(statistic=-15.439826683525418, pvalue=8.750571156437711e-34),
 Ttest_1sampResult(statistic=1.4845341263724807, pvalue=0.1395867786115413),
 Ttest_1sampResult(statistic=-13.659513100277254, pvalue=1.5157657361710158e-28),
 Ttest_1sampResult(statistic=13.7323961384641, pvalue=1.5754742985301713e-28),
 Ttest_1sampResult(statistic=13.057014526

In [107]:
rep.describe()

,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
count,165.000000,148.000000,164.000000,165.000000,165.000000,166.000000,162.000000,157.000000,165.000000,165.000000,159.000000,155.000000,158.000000,161.000000,156.000000,146.000000
mean,0.187879,0.506757,0.134146,0.987879,0.951515,0.897590,0.240741,0.152866,0.115152,0.557576,0.132075,0.870968,0.860759,0.981366,0.089744,0.657534
std,0.391804,0.501652,0.341853,0.109760,0.215442,0.304104,0.428859,0.361010,0.320176,0.498186,0.339643,0.336322,0.347298,0.135649,0.286735,0.476168
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
75%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [108]:
dem.describe()

,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
count,258.000000,239.000000,260.000000,259.000000,255.000000,258.000000,259.000000,263.000000,248.000000,263.000000,255.000000,249.000000,252.000000,257.000000,251.000000,185.000000
mean,0.604651,0.502092,0.888462,0.054054,0.215686,0.476744,0.772201,0.828897,0.758065,0.471483,0.505882,0.144578,0.289683,0.350195,0.637450,0.935135
std,0.489876,0.501045,0.315405,0.226562,0.412106,0.500430,0.420224,0.377317,0.429121,0.500138,0.500949,0.352383,0.454518,0.477962,0.481697,0.246956
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [109]:
#Null hypothesis of 1 for the rep's data
[[one_sample_ttest(a, 1) for a in rep_data]]

[[Ttest_1sampResult(statistic=-26.625236633811387, pvalue=1.978873197183477e-61),
  Ttest_1sampResult(statistic=-1.4185450076223511, pvalue=0.1579292482594923),
  Ttest_1sampResult(statistic=-11.961605243444543, pvalue=1.8656648229239887e-23),
  Ttest_1sampResult(statistic=-32.43595087385152, pvalue=5.293293090366981e-73),
  Ttest_1sampResult(statistic=-2.890793645020198, pvalue=0.004363402589282088),
  Ttest_1sampResult(statistic=-4.338836636208457, pvalue=2.488389920449274e-05),
  Ttest_1sampResult(statistic=-22.533735393166197, pvalue=1.1791229999687983e-51),
  Ttest_1sampResult(statistic=-29.402380855978315, pvalue=1.646323339608132e-65),
  Ttest_1sampResult(statistic=-35.49944402826317, pvalue=6.985438779792963e-79),
  Ttest_1sampResult(statistic=-11.407472760546423, pvalue=1.4608897556395605e-22),
  Ttest_1sampResult(statistic=-32.22244115962839, pvalue=2.3835416257429457e-71),
  Ttest_1sampResult(statistic=-4.776485613378817, pvalue=4.122594240061259e-06),
  Ttest_1sampResult(st

In [110]:
#Null hypothesis of 1 for the dem's data
[[one_sample_ttest(a, 1) for a in dem_data]]

[[Ttest_1sampResult(statistic=-12.96296499796484, pvalue=6.590394568934029e-30),
  Ttest_1sampResult(statistic=-67.19374970932937, pvalue=1.7479453896049469e-165),
  Ttest_1sampResult(statistic=-15.36283393995609, pvalue=1.8031537722768159e-37),
  Ttest_1sampResult(statistic=-5.702205846437985, pvalue=3.217258173105712e-08),
  Ttest_1sampResult(statistic=-30.39138633949369, pvalue=1.4023105444477201e-86),
  Ttest_1sampResult(statistic=-16.7950341092749, pvalue=3.12862805821349e-43),
  Ttest_1sampResult(statistic=-8.724104538575864, pvalue=3.377871549159516e-16),
  Ttest_1sampResult(statistic=-7.354085178140069, pvalue=2.464384519854037e-12),
  Ttest_1sampResult(statistic=-8.87861403790268, pvalue=1.418109388321789e-16),
  Ttest_1sampResult(statistic=-17.137489559066015, pvalue=1.1309092327824875e-44),
  Ttest_1sampResult(statistic=-15.750968962442089, pvalue=1.839716417353916e-39),
  Ttest_1sampResult(statistic=-38.305787204198445, pvalue=3.89658572232871e-106),
  Ttest_1sampResult(sta

## Stretch Goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.
### Start with a 1-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then try a 2-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://lh3.googleusercontent.com/proxy/rJJ5ZOL9ZDvKOOeBihXoZDgfk7uv1YsRzSQ1Tc10RX-r2HrRpRLVqlE9CWX23csYQXcTniFwlBg3H-qR8MKJPBGnjwndqlhDX3JxoDE5Yg) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then check your Answers using Scipy!